In [1]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio
import asyncio
nest_asyncio.apply()

In [ ]:
# from rags import get_router_query_engine

# file_path = "./datasets/lora.pdf"
# query = "What are LOW-RANK-PARAMETRIZED UPDATE MATRICES?"

# async def main():
#     response = await get_router_query_engine(file_path, query)
#     print(response)

# # Run the async function
# await main()

In [3]:
from rags import get_router_query_engine

file_path = "./datasets/lora.pdf"
query = "What are LOW-RANK-PARAMETRIZED UPDATE MATRICES?"

response = await get_router_query_engine(file_path, query)

print (str(response))

c:\Users\Admin\AppData\Local\pypoetry\Cache\virtualenvs\rags-agent-h2EaaEnM-py3.12\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


=== Calling Function ===
Calling function: summary_tool with args: {"input": "LOW-RANK-PARAMETRIZED UPDATE MATRICES"}
=== Function Output ===
The low-rank parametrized update matrices approach involves constraining weight updates to have a low "intrinsic rank," optimizing rank decomposition matrices for training dense layers indirectly while keeping pre-trained weights frozen. This method significantly reduces trainable parameters, leading to efficient adaptation for downstream tasks. It also reduces memory and storage usage, enabling easier task-switching and lower hardware requirements. Additionally, it ensures no additional inference latency compared to fully fine-tuned models. The approach involves amplifying task-specific directions within the model for downstream task adaptation, exploring the correlation between different layers, the effect of varying the rank parameter, and the amplification factor for task-specific directions in the update matrices. The study indicates that th

TypeError: object AgentChatResponse can't be used in 'await' expression